In [19]:
import logging

from Loader import fileloader_proto as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
from Loader import dfutils as utils
import pandas as pd
import numpy as np
import posixpath
from datetime import datetime
import time
import xlwings as xw
import copy

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

**Ingresos Nuevos**
----------
Ejecutar el siguiente codigo como requisito para las siguientes secciones de Ingresos Nuevos

In [26]:
month="201904"
inifile = fl.ReadIniFile(mercado="empresas")
parser = inifile.getIniFileParser()

# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)

# File :  _RRHH Ingresos
section_ingresos = fl.SectionObj(inifile,"Ingresos")
# Guardamos el nombre de nuestro archivo de nuevos ingresos
comis_ingresos_archivo_nombre = section_ingresos.getParameter('filelist')[0]
# Obtenemos las columnas que deberia tener el reporte de ingresos nuevos.
cols_required = section_ingresos.getParameter('cols')

loader1 = fl.LoadFileProcess(section_ingresos)
ingresos = loader1.loadFile()

INFO:    Values loaded from globals.ini:
INFO:     * mainpath C:/Users/Johnny/OneDrive - Entel Peru S.A
INFO:     * scriptspath C:/Users/Johnny/Documents/Comisiones/Comisiones-python
INFO:     * databasepath C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB
INFO:    datapath value is C:/Users/Johnny/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
INFO:    testpath value is C:/Users/Johnny/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/test
INFO:    Setting database path to C:/Users/Johnny/OneDrive - Entel Peru S.A/SQLiteDB/mercado_empresas_db.sqlite


***<span style="color:red">Copiar</span>*** Ingresos Nuevos del Archivo de Atracción
-----

**Requisitos**: 
- Haber filtrado las personas de interes en el archivo que envia attracion
- Asegurarse la hoja se llame BASE en el archivo de ingresos nuevos de comisiones
- Asegurarse que los nombres de columnas del archivo de atraccion no hayan cambiado

**Pasos**:
- En el archivo de atracción filtrar solo las personas que comisionan, eliminar las demás
- Ejecutar script
- Convertir la columna DNI a formato Texto
- Jalar las fórmulas
- Proseguir con la subida a la base de datos de los nuevos ingresos





In [64]:
#Obtenemos la lista de los nombres de los libros de Excel abiertos.
wb_names = utils.getOpenedWorkbooksNames()
#Seleccionamos el archivo que contenga "REPORTEREMUNERACIONES"
keyname = "REPORTEREMUNERACIONES"
atraccion_ingresos_filename = [s for s in wb_names if keyname in s][0]
#Nos conectamos al archivo y hoja de calculo enviado por atraccion
wb_ingresos = xw.Book(atraccion_ingresos_filename)
sht = wb_ingresos.sheets[0]
#Nos posicionamos en la celda A1 extendemos hacia la esquina opuesta retornando el contenido como dataframe
df = sht.range('A1').options(pd.DataFrame,
                         index=False, expand='table').value
#Filtramos las columnas que debe tener el archivo de atraccion.
cols_required = cols_required[:-3] + ['Variable', 'VP', 'CC', 'Jefe DIrecto']
#Obtenemos el dataframe a ser pegado en nuestro archivo de comisiones.
df_final = df[cols_required]

#Nos conectamos a nuestro archivo de Ingresos
wb_comis_ingresos = xw.Book(comis_ingresos_archivo_nombre)
sheet_comis_ingresos = wb_comis_ingresos.sheets('BASE')
#Obtener el nro. de fila del último registro
lastrow = sheet_comis_ingresos.api.Cells(65536, 1).End(xw.constants.Direction.xlUp).Row
#Pegamos el dataframe
sheet_comis_ingresos.range((lastrow+1, 1)).options(header=False, index=False).value = df_final


Insertar ***<span style="color:red">Nuevos</span>*** Ingresos en la BD
-----

In [33]:
# Separamos los que se van a agregar a la base de datos y los que unicamente se va actualizar su CODIGO_INAR
ingresos_add = ingresos[ingresos.ADD_TO_DB == "SI"]
ingresos_add.drop('ADD_TO_DB', axis=1, inplace=True)
ingresos_add.drop('UPDATE_CODIGO_INAR', axis=1, inplace=True)
ingresos_add


El tamaño de Ingresos es 32 registros


,DNI,NOMBRES,APELLIDO_PATERNO,POSICION_EMPL,FECHA_INGRESO,CODIGO_INAR
0,45381023,PEDRO,CRUZ ROMERO,EJECUTIVO INSIDE SALES,2019-05-16,0.0
1,42794664,OSCAR MANUEL,DE-LA-CRUZ MEDINA,EJECUTIVO INSIDE SALES,2019-05-16,NaN
2,45478650,MANUEL ALEXANDER,FIGUEROA RIMACHE,EJECUTIVO INSIDE SALES,2019-05-16,NaN
3,41242865,DIANA PATRICIA,GUTIERREZ VELASQUEZ,EJECUTIVO INSIDE SALES,2019-05-16,NaN
4,70909667,FELIX JHONATAN,PASACHE INGA,EJECUTIVO INSIDE SALES,2019-05-16,NaN
5,41317674,ANTONIO FELIPE,QUISPE ARIAS,EJECUTIVO INSIDE SALES,2019-05-16,NaN
6,43323015,ROSSEMARY,QUISPE REYES,EJECUTIVO INSIDE SALES,2019-05-16,NaN
7,43598156,GONZALO AUGUSTO,VALDIVIA MOROTE,CONSULTOR DE SERVICIO VALOR AGREGADO,2019-05-16,NaN
8,42969746,PAREDES SAENZ,VICTOR LUIS,EJECUTIVO DE ADMINISTRACIÓN PÚBLICA DE CORPORA...,2019-04-01,NaN
9,41179299,DIAZ AMAYO,FERNANDO ALONSO,CONSULTOR DE NEGOCIOS REGIONES,2019-05-01,NaN


**Ejecutar lo siguiente para modificar la BD**

In [22]:
dbmanager.dbOperation('insert','tblEmpleados',ingresos_add)

Los registros de la tabla tblEmpleados es 31 registros insertados 


Actualizar ***<span style="color:red">Codigo Inar</span>*** Ingresos en la BD
----

In [34]:
ingresos_update = ingresos[ingresos.UPDATE_CODIGO_INAR == "SI"]
ingresos_update.drop('ADD_TO_DB', axis=1, inplace=True)
ingresos_update.drop('UPDATE_CODIGO_INAR', axis=1, inplace=True)
ingresos_update

,DNI,NOMBRES,APELLIDO_PATERNO,POSICION_EMPL,FECHA_INGRESO,CODIGO_INAR
0,45381023,PEDRO,CRUZ ROMERO,EJECUTIVO INSIDE SALES,2019-05-16,0.0
1,42794664,OSCAR MANUEL,DE-LA-CRUZ MEDINA,EJECUTIVO INSIDE SALES,2019-05-16,NaN
2,45478650,MANUEL ALEXANDER,FIGUEROA RIMACHE,EJECUTIVO INSIDE SALES,2019-05-16,NaN
3,41242865,DIANA PATRICIA,GUTIERREZ VELASQUEZ,EJECUTIVO INSIDE SALES,2019-05-16,NaN
4,70909667,FELIX JHONATAN,PASACHE INGA,EJECUTIVO INSIDE SALES,2019-05-16,NaN
5,41317674,ANTONIO FELIPE,QUISPE ARIAS,EJECUTIVO INSIDE SALES,2019-05-16,NaN
6,43323015,ROSSEMARY,QUISPE REYES,EJECUTIVO INSIDE SALES,2019-05-16,NaN
7,43598156,GONZALO AUGUSTO,VALDIVIA MOROTE,CONSULTOR DE SERVICIO VALOR AGREGADO,2019-05-16,NaN
8,42969746,PAREDES SAENZ,VICTOR LUIS,EJECUTIVO DE ADMINISTRACIÓN PÚBLICA DE CORPORA...,2019-04-01,NaN
9,41179299,DIAZ AMAYO,FERNANDO ALONSO,CONSULTOR DE NEGOCIOS REGIONES,2019-05-01,NaN


**Ejecutar lo siguiente para modificar la BD**

In [35]:
dbmanager.dbOperation('update','UpdateEmpleadoCodigoInar',ingresos_update)

Los registros de la tabla UpdateEmpleadoCodigoInar es 31 registros actualizados 


Actualizar ***<span style="color:red">Puestos</span>*** Ingresos en la BD
-----

In [23]:
section_ingresos_actualizar = fl.SectionObj(inifile,"Empleados_Update_Puestos")
loader2 = fl.LoadFileProcess(section_ingresos_actualizar)
ingresos_camb_puesto = loader2.loadFile()

El tamaño de Empleados_Update_Puestos es 1 registros


**Ejecutar lo siguiente para modificar la BD**

In [ ]:
dbmanager.dbOperation('update','UpdateEmpleadoCodigoInar',ingresos_camb_puesto)